In [19]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,mean_absolute_error
from tensorflow.keras.utils import plot_model
from tensorflow import constant_initializer
import matplotlib.pyplot as plt
import json
import shutil
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import sys
sys.path.append("../code/")
from model import PhasedSNForecastProbabilisticIntervalModel
out_steps = 3

In [21]:
def normalize(data, min_val = None, max_val = None):
    masked_data = np.ma.masked_where(data < 0, data)
    min_val = np.tile([0,0,0], (len(data),1)) if min_val is None else min_val
    max_val = np.tile([15, 23, 1],(len(data),1)) if max_val is None else max_val
    
    for i in range(masked_data.shape[1]):
        masked_data.data[:,i,:] = (masked_data.data[:,i,:] - min_val)/(max_val-min_val)
    
    return_data = masked_data.data
    return_data[masked_data.mask] = -1
    return return_data, min_val, max_val
    
def denormalize(data, min_val, max_val):
    masked_data = np.ma.masked_where(data < 0, data)
    
    for i in range(masked_data.shape[1]):
        masked_data.data[:,i,:] = (masked_data.data[:,i,:] * (max_val-min_val))  +  min_val
    
    return_data = masked_data.data
    return_data[masked_data.mask] = -1
    return return_data

In [22]:
data = np.load("../data/padded_x_train.npy")
len_data = data.shape[1]
X_train, y_train = data[:,:-out_steps,:],  data[:,-out_steps:,:]
X_train, data_min_val, data_max_val = normalize(X_train)
y_train, _, _ = normalize(y_train,min_val=data_min_val, max_val=data_max_val)

In [23]:
data_val = np.load("../data/padded_x_val.npy")
len_data = data_val.shape[1]
X_val, y_val = data_val[:,:-out_steps,:],  data_val[:,-out_steps:,:]
X_val, data_val_min_val, data_val_max_val = normalize(X_val)
y_val, _, _ = normalize(y_val,min_val=data_val_min_val,max_val=data_val_max_val)

In [24]:
inputs = X_train
outputs = y_train
inputs_val = X_val
outputs_val = y_val

outputs = {}
outputs_val = {}

outputs["prediction"] = y_train
outputs_val["prediction"] = y_val

for interval in ["upper", "lower"]:
    outputs[interval] = np.expand_dims(y_train[:,:,1],axis=-1)
    outputs_val[interval] = np.expand_dims(y_val[:,:,1],axis=-1)

In [25]:
class SaveData(tf.keras.callbacks.Callback):
    def __init__(self,logdir, keys,**kwargs):
        super().__init__(**kwargs)
        self.file_writer = tf.summary.create_file_writer(logdir + "/metrics")
        self.file_writer.set_as_default()
        self.keys = keys
        
    def on_epoch_end(self, epoch, logs=None):
        for key in self.keys:
            tf.summary.scalar(key, data=logs.get(key), step=epoch)

In [26]:
import datetime
#Early stops
early_stop = tf.keras.callbacks.EarlyStopping( monitor='val_loss', min_delta=1e-10, patience=10)

#Tensorboard
logdir = "../data/training/logs/PI" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = tf.keras.callbacks.TensorBoard(logdir)
saver = SaveData(logdir, ["PICW"])
shutil.rmtree("../data/training/logs/PI",ignore_errors=True)


#Checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint("../data/training_PI/model_checkpoints/checkpoint", monitor='val_loss', verbose=0, save_best_only=True)

callbacks = [tensorboard,checkpoint, early_stop, saver]

In [27]:
#Loading and preparing model
from model import PhasedSNForecastModel
base_model = PhasedSNForecastModel(units=64, out_steps=out_steps,features = 3)
base_model.compile(optimizer="rmsprop", loss="mse")
_ = base_model.fit(X_train[:2], y_train[:2])


base_model.load_weights("../data/sn_model_small.h5")

1/1 [==============================] - 0s 834us/step - loss: 0.2766


In [35]:
from tensorflow_addons.utils.keras_utils import LossFunctionWrapper
from tensorflow_addons.utils.types import TensorLike, FloatTensorLike
from typeguard import typechecked

@tf.function
def custom_pinball_loss(y_true: TensorLike, y_pred: TensorLike, tau: FloatTensorLike = 0.5) -> tf.Tensor:
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    
    
    # Broadcast the pinball slope along the batch dimension
    tau = tf.expand_dims(tf.cast(tau, y_pred.dtype), 0)
    one = tf.cast(1, tau.dtype)

    pinball = tf.where(y_pred > y_true, tau * (y_pred - y_true), (1-tau) * (y_true-y_pred) )
    return tf.reduce_mean(pinball, axis=-1)

class CustomPinballLoss(LossFunctionWrapper):
    @typechecked
    def __init__(
        self,
        tau: FloatTensorLike = 0.5,
        reduction: str = tf.keras.losses.Reduction.AUTO,
        name: str = "custom_pinball_loss",
    ):
        super().__init__(custom_pinball_loss, reduction=reduction, name=name, tau=tau)
        
        
@tf.function
def inverse_pinball_loss(y_true: TensorLike, y_pred: TensorLike, tau: FloatTensorLike = 0.5) -> tf.Tensor:
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    
    
    # Broadcast the pinball slope along the batch dimension
    tau = tf.expand_dims(tf.cast(tau, y_pred.dtype), 0)
    one = tf.cast(1, tau.dtype)

    pinball = tf.where(y_pred > y_true, (1-tau) * (y_pred - y_true), tau * (y_true-y_pred) )
    return tf.reduce_mean(pinball, axis=-1)   

class InversePinballLoss(LossFunctionWrapper):
    @typechecked
    def __init__(
        self,
        tau: FloatTensorLike = 0.5,
        reduction: str = tf.keras.losses.Reduction.AUTO,
        name: str = "inverse_pinball_loss",
    ):
        super().__init__(inverse_pinball_loss, reduction=reduction, name=name, tau=tau)
        
        

In [47]:
model = PhasedSNForecastProbabilisticIntervalModel(units=300, out_steps=out_steps, model = base_model, dropout=0.0)
model.rnn.trainable = False
model.denses.trainable = False
model.cells.trainable = False

alpha = 0.30
losses = {
    "prediction": None,
    "lower": CustomPinballLoss(tau=(alpha/2), reduction=tf.keras.losses.Reduction.NONE),
    "upper": CustomPinballLoss(tau=1-(alpha/2), reduction=tf.keras.losses.Reduction.NONE)
}
model.compile(optimizer="rmsprop", loss=losses)

In [54]:
MAX_EPOCHS=1000
history = model.fit(inputs,outputs,
                    batch_size=150, 
                    epochs=MAX_EPOCHS, 
                    validation_data=(inputs_val,outputs_val), 
                    callbacks=callbacks)

Epoch 1/1000
30/30 [==============================] - 2s 72ms/step - loss: 0.2903 - lower_loss: 0.0690 - upper_loss: 0.0505 - PICW: 0.1708 - val_loss: 0.3393 - val_lower_loss: 0.0951 - val_upper_loss: 0.0618 - val_PICW: 0.1824
Epoch 2/1000
30/30 [==============================] - 2s 70ms/step - loss: 0.2186 - lower_loss: 0.0609 - upper_loss: 0.0416 - PICW: 0.1156 - val_loss: 0.0620 - val_lower_loss: 0.0282 - val_upper_loss: 0.0149 - val_PICW: 0.0189
Epoch 3/1000
30/30 [==============================] - 2s 71ms/step - loss: 0.2781 - lower_loss: 0.0700 - upper_loss: 0.0534 - PICW: 0.1542 - val_loss: 0.0953 - val_lower_loss: 0.0376 - val_upper_loss: 0.0170 - val_PICW: 0.0407
Epoch 4/1000
30/30 [==============================] - 2s 71ms/step - loss: 0.1914 - lower_loss: 0.0531 - upper_loss: 0.0380 - PICW: 0.1000 - val_loss: 0.0564 - val_lower_loss: 0.0260 - val_upper_loss: 0.0149 - val_PICW: 0.0154
Epoch 5/1000
30/30 [==============================] - 2s 71ms/step - loss: 0.3146 - lower_lo

In [55]:
history_dict = history.history
json.dump(history_dict, open("../data/training_PI/history_model.json", 'w'))

In [56]:
model.save_weights("../data/sn_model_PI_small.h5")

In [57]:
data_test = np.load("../data/padded_x_test.npy")[:,:,:]
# data_test, data_test_min_val, data_test_max_val = normalize(data_test)
X_test, y_test = data_test[:,:-out_steps,:], data_test[:,-out_steps:, :]
X_test, data_test_min_val, data_test_max_val = normalize(X_test)
y_test, _, _ = normalize(y_test,min_val=data_test_min_val, max_val=data_test_max_val)

#Doing inference on Train data
y_hat_train = model.predict(X_train)
#Denormalizing train
dX_train = denormalize(X_train, data_min_val,data_max_val)
dy_hat_train = {}
dy_hat_train["prediction"] = denormalize(y_hat_train["prediction"], data_min_val,data_max_val)
for key in ["upper", "lower"]:
    dy_hat_train[key] = denormalize(y_hat_train[key], data_min_val[:,1][:,np.newaxis],data_max_val[:,1][:,np.newaxis])
dy_train = denormalize(y_train, data_min_val,data_max_val)

# Doing inference on Test data
y_hat = model.predict(X_test)
# Denormalizing results
dX_test = denormalize(X_test, data_test_min_val,data_test_max_val)
dy_hat = {}
dy_hat["prediction"] = denormalize(y_hat["prediction"],data_test_min_val,data_test_max_val) 
for key in ["upper", "lower"]:
    dy_hat[key] = denormalize(y_hat[key],data_test_min_val[:,1][:,np.newaxis],data_test_max_val[:,1][:,np.newaxis])
dy_test = denormalize(y_test,data_test_min_val,data_test_max_val)

In [60]:
import os
def plot_data(x, y_real, y_hat, sample=0, save=False, path=""):
    plt.figure(figsize=(12,6))
    plt.gca().invert_yaxis()

    xx = x[sample]
    x_slice = np.where(~(xx[:,1] < 0) )[0]
    unpadded_xx = xx[x_slice,:]
    xx_time = unpadded_xx
    xx_time[:,0] = np.cumsum(xx_time[:,0]) 
    last_time = xx_time[:,0][-1]

    y_real_sample = y_real[sample]
    y_real_sample[:,0] = np.cumsum(y_real_sample[:,0]) + last_time

    y_hat_sample = y_hat["prediction"][sample]
    y_hat_sample[:,0] = np.cumsum(y_hat_sample[:,0]) + last_time

    plt.scatter(xx_time[:,0], xx_time[:,1], label="History")
    plt.scatter(y_real_sample[:,0], y_real_sample[:,1], label="Real")
    plt.scatter(y_hat_sample[:,0], y_hat_sample[:,1], label="Prediction")
    plt.fill_between(y_hat_sample[:,0], y_hat["lower"][sample,:,0], y_hat["upper"][sample,:,0], alpha=0.2, label="Pinball Loss Q(0.15,0.85)")
    plt.xlabel("Time $mjd-\min(mjd)$")
    plt.ylabel("Mag")
    plt.legend()
    if save:
        plt.savefig(os.path.join(path, f"{str(sample).rjust(5,'0')}.png"))
        plt.cla()
        plt.clf()
        plt.close()


f = lambda sample: plot_data(dX_test, dy_test, dy_hat,sample=sample)
interact(f, sample=(0,len(dX_test)-1))

interactive(children=(IntSlider(value=728, description='sample', max=1456), Output()), _dom_classes=('widget-i…

<function __main__.<lambda>(sample)>

In [59]:
# import os
# import progressbar
# bar = progressbar.ProgressBar(max_value=len(X_test))
# os.makedirs("../data/plots_test_PI/",exist_ok=True)

# x = dX_test
# y_real = dy_test
# y_hat = dy_hat
# bar.start()
# for sample in range(len(dX_test)):
#     plt.figure(figsize=(12,6))
#     plt.gca().invert_yaxis()
#     x_masked = np.ma.masked_where(x < 0, x)
#     plt.scatter(x_masked[sample,:,0], x_masked[sample,:,1], label="History")
#     plt.scatter(y_real[sample,:,0], y_real[sample,:,1], label="Real")
#     plt.scatter(y_hat["prediction"][sample,:,0], y_hat["prediction"][sample,:,1], label="Prediction")
#     plt.fill_between(y_hat["prediction"][sample,:,0], y_hat["lower"][sample,:,0], y_hat["upper"][sample,:,0], alpha=0.2)
#     plt.xlabel("Time $mjd-\min(mjd)$")
#     plt.ylabel("Mag")
#     plt.savefig(f"../data/plots_test_PI/{str(sample).rjust(5,'0')}")
#     plt.clf()
#     plt.cla()
#     plt.close()
#     bar.update(sample+1)